<a href="https://colab.research.google.com/github/Santiann/Trabalho-Final-Previs-o-de-pre-o-de-voo/blob/main/Previs%C3%A3o_de_pre%C3%A7o_de_voo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando todas as bibliotecas necessárias
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.svm import SVR
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# Define o dataset e o destino do download
dataset = "shubhambathwal/flight-price-prediction"
download_path = "datasets/"

# Comando para baixar o dataset
os.system(f"kaggle datasets download -d {dataset} -p {download_path} --unzip")

print(f"Dataset baixado e extraído em: {download_path}")

In [ ]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Vamos ver o que há nos dados
df=pd.read_csv('/content/datasets/Clean_Dataset.csv')
df.head()

In [ ]:
# Eliminando a coluna inútil 'Unnamed: 0'
df=df.drop('Unnamed: 0',axis=1)

In [ ]:
# Uma informação rápida sobre os dados
df.info()

In [ ]:
# Descrição Estatística dos Dados
df.describe()

In [ ]:
# Seleciona apenas colunas numéricas
df_numeric = df.select_dtypes(include=['number'])

# Calcula a correlação e transposta
correlation_matrix = df_numeric.corr().T

print(correlation_matrix)

In [ ]:
# Tamanho dos dados
df.shape

In [ ]:
df1=df.groupby(['flight','airline'],as_index=False).count()
df1.airline.value_counts()

In [ ]:
# Configurando o estilo e ajustando o tamanho da figura
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Criando o gráfico com uma paleta mais vibrante
palette = sns.color_palette("coolwarm", len(df1['airline'].unique()))
ax = sns.countplot(data=df1, x='airline', palette=palette, order=df1['airline'].value_counts().index)

# Adicionando rótulos de valores nas barras
for p in ax.patches:
    ax.annotate(f'{p.get_height()}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                fontsize=12, color='black',
                xytext=(0, 7), textcoords='offset points')

# Melhorando os títulos e rótulos
plt.title('Contagem de Voos por Companhia Aérea', fontsize=18, fontweight='bold')
plt.xlabel('Companhia Aérea', fontsize=14)
plt.ylabel('Número de Voos', fontsize=14)

# Girando os rótulos no eixo X (caso necessário)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

# Exibindo o gráfico
plt.tight_layout()
plt.show()

In [ ]:
df2=df.groupby(['flight','airline','class'],as_index=False).count()
df2['class'].value_counts()

In [ ]:
# Configurando o tamanho da figura
plt.figure(figsize=(8, 8))

# Criando o gráfico de pizza com melhorias
colors = sns.color_palette("cool", len(df2['class'].unique()))  # Paleta de cores
df2['class'].value_counts().plot(
    kind='pie',
    textprops={'color': 'black', 'fontsize': 12},
    autopct='%1.1f%%',
    startangle=140,
    colors=colors,
    wedgeprops={'edgecolor': 'black', 'linewidth': 1}
)

# Adicionando título
plt.title('Distribuição das Classes de Voos', fontsize=16, fontweight='bold')

# Melhorando a legenda
plt.legend(
    labels=df2['class'].value_counts().index,
    title='Classes',
    title_fontsize=12,
    loc='upper right',
    fontsize=12,
    frameon=True,
    bbox_to_anchor=(1.3, 0.8)
)

# Ajustando o layout para evitar sobreposição
plt.tight_layout()

# Exibindo o gráfico
plt.show()

In [ ]:
# Calculando a média dos preços por companhia
mean_prices = df.groupby('airline')['price'].mean().sort_values()

# Configurando o estilo e tamanho do gráfico
sns.set_style("whitegrid")
plt.figure(figsize=(16, 6))

# Criando o gráfico de barras
sns.barplot(
    x=mean_prices.index,
    y=mean_prices.values,
    palette='coolwarm'
)

# Adicionando rótulos de valores acima das barras
for i, val in enumerate(mean_prices.values):
    plt.text(i, val + 10, f'${val:.2f}', ha='center', fontsize=12, color='black', weight='bold')

# Ajustando títulos e rótulos
plt.title('Média de Preços por Companhia Aérea', fontsize=18, fontweight='bold')
plt.xlabel('Companhia Aérea', fontsize=14)
plt.ylabel('Preço Médio da Passagem (em $)', fontsize=14)

# Ajustando rótulos do eixo X
plt.xticks(rotation=45, fontsize=12)

# Ajustando espaçamento
plt.tight_layout()

# Exibindo o gráfico
plt.show()

In [ ]:
# Grafico para visualizar a concentração de preços por classes
plt.figure(figsize=(12, 6))
sns.stripplot(x='class', y='price', data=df, palette='viridis', size=7, jitter=True)

# Melhorando o título e os rótulos
plt.title('Distribuição de Preços por Classe', fontsize=18, fontweight='bold')
plt.xlabel('Classe do Bilhete', fontsize=15, labelpad=10)
plt.ylabel('Preço do Bilhete (USD)', fontsize=15, labelpad=10)

# Ajustando o estilo
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar o gráfico
plt.tight_layout()
plt.show()

In [ ]:
# Grafico que mostra a quantidade de bilhetes vendidos por números de paradas,
# dando a entender que passagens com pelo menos 1 paradas são mais caras do que viagens sem nenhuma parada.
# Dando a entender que paradas são mais buscadas que viagens sem paradas.
plt.figure(figsize=(12, 6))

# Calculando as médias
mean_prices = df.groupby('stops')['price'].mean().reset_index()

# Gráfico de linha
sns.lineplot(x='stops', y='price', data=mean_prices, marker='o', color='blue', linewidth=2.5)

# Melhorando os rótulos e título
plt.title('Preço Médio do Bilhete por Número de Paradas', fontsize=18, fontweight='bold')
plt.xlabel('Número de Paradas', fontsize=15, labelpad=10)
plt.ylabel('Preço Médio (USD)', fontsize=15, labelpad=10)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(24, 10))

# Gráfico para Horário de Partida vs Preço da Passagem
plt.subplot(1, 2, 1)
sns.violinplot(x='departure_time', y='price', data=df, inner="quartile", scale="width", palette="muted")
plt.title('Horário de Partida vs Preço da Passagem', fontsize=20)
plt.xlabel('Horário de Partida', fontsize=15)
plt.ylabel('Preço (R$)', fontsize=15)

# Gráfico para Horário de Chegada vs Preço da Passagem
plt.subplot(1, 2, 2)
sns.violinplot(x='arrival_time', y='price', data=df, inner="quartile", scale="width", palette="husl")
plt.title('Horário de Chegada vs Preço da Passagem', fontsize=20)
plt.xlabel('Horário de Chegada', fontsize=15)
plt.ylabel('Preço (R$)', fontsize=15)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(24,10))

# Gráfico de violino para cidade de origem
plt.subplot(1, 2, 1)
sns.violinplot(x='source_city', y='price', data=df, inner="quartile", scale="width", palette='muted')
plt.title('Cidade de Origem Vs Preço da Passagem', fontsize=20)
plt.xlabel('Cidade de Origem', fontsize=15)
plt.ylabel('Preço', fontsize=15)

# Gráfico de violino para cidade de destino
plt.subplot(1, 2, 2)
sns.violinplot(x='destination_city', y='price', data=df, inner="quartile", scale="width", palette='husl')
plt.title('Cidade de Destino Vs Preço da Passagem', fontsize=20)
plt.xlabel('Cidade de Destino', fontsize=15)
plt.ylabel('Preço', fontsize=15)

plt.show()

In [ ]:
# Estilo do fundo
plt.style.use('dark_background')

# Configurações da figura
plt.figure(figsize=(20, 8))

# Paleta de cores personalizada
sns.set_palette("coolwarm")

# Gráfico com linhas suaves e transparência
sns.lineplot(
    data=df,
    x='duration',
    y='price',
    hue='class',
    palette='coolwarm',
    linewidth=2,
    alpha=0.8,
    marker='o'  # Marcadores para diferenciar classes
)

# Melhorias no título e rótulos
plt.title(
    'Preço das Passagens vs Duração do Voo por Classe',
    fontsize=22,
    pad=20,  # Espaçamento
    fontweight='bold'
)
plt.xlabel('Duração (horas)', fontsize=16, labelpad=15)
plt.ylabel('Preço (R$)', fontsize=16, labelpad=15)

# Adicionar grid discreto
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

# Ajustar posição da legenda
plt.legend(title='Classe', fontsize=14, title_fontsize=16, loc='upper left')

# Mostrar o gráfico
plt.show()

In [ ]:
# Estilo do fundo
plt.style.use('dark_background')

# Configurações da figura
plt.figure(figsize=(20, 8))

# Gráfico com linha mais visível e suavidade
sns.lineplot(
    data=df,
    x='days_left',
    y='price',
    color='#51b6ff',  # Azul mais claro e agradável
    linewidth=2,
    alpha=0.8
)

# Melhorias no título e rótulos
plt.title(
    'Dias Restantes para Partida vs Preço das Passagens',
    fontsize=22,
    pad=20,  # Espaçamento
    fontweight='bold'
)
plt.xlabel('Dias Restantes para a Partida', fontsize=16, labelpad=15)
plt.ylabel('Preço (R$)', fontsize=16, labelpad=15)

# Adicionar grade discreta
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

# Ajustar margens para melhor exibição
plt.tight_layout()

# Mostrar o gráfico
plt.show()


In [ ]:
# Configuração de estilo
sns.set_style("whitegrid")

plt.figure(figsize=(20, 8))

# Gráfico de linhas com paleta e legendas
sns.lineplot(
    data=df,
    x='days_left',
    y='price',
    hue='airline',
    palette='hls',
    linewidth=2.5
)

# Títulos e legendas traduzidos
plt.title('Dias Restantes para Partida vs Preço da Passagem por Companhia Aérea', fontsize=18, fontweight='bold')
plt.xlabel('Dias Restantes para Partida', fontsize=15)
plt.ylabel('Preço da Passagem (R$)', fontsize=15)

# Ajustando a legenda
plt.legend(title='Companhia Aérea', fontsize=12, title_fontsize=14, loc='upper right')

# Ajuste dos limites e exibição
plt.tight_layout()
plt.show()


In [ ]:
# Primeiro agrupamento: Agrupa o DataFrame por 'flight', 'source_city', 'destination_city', 'airline' e 'class'.
# O parâmetro 'as_index=False' garante que os grupos não sejam usados como índice no resultado.
# Conta o número de ocorrências em cada grupo criado anteriormente.
# Segundo agrupamento: Reagrupa os dados resultantes, agora por 'source_city' e 'destination_city'.
# O parâmetro 'as_index=False' novamente impede que os grupos sejam usados como índice no resultado.
# Conta o número de voos únicos ('flight') para cada combinação de 'source_city' e 'destination_city'.
# Seleciona apenas as 10 primeiras linhas do resultado.
# Quantos voos diferentes existem entre pares de cidades de origem e destino, considerando apenas as 10 primeiras combinações?

df.groupby(['flight', 'source_city', 'destination_city', 'airline', 'class'], as_index=False).count().groupby(['source_city', 'destination_city'], as_index=False)['flight'].count().head(10)

In [ ]:
# Agrupa o DataFrame pelas colunas 'airline', 'source_city' e 'destination_city'.
# O parâmetro 'as_index=False' garante que os grupos não sejam usados como índice no resultado.
# Calcula a média dos preços ('price') para cada grupo definido anteriormente.
# Seleciona as 10 primeiras linhas do resultado.
# Qual é o preço médio dos voos por companhia aérea entre pares de cidades de origem e destino, considerando apenas as 10 primeiras combinações?

df.groupby(['airline', 'source_city', 'destination_city'], as_index=False)['price'].mean().head(10)

In [ ]:
# Criando um arquivo de backup
df_bk=df.copy()

In [ ]:
# Convertendo os rótulos em um formato numérico usando o Label Encoder
# Converte colunas categórias para númericas
le=LabelEncoder()
for col in df.columns:
    if df[col].dtype=='object':
        df[col]=le.fit_transform(df[col])

In [ ]:
# armazenando as variáveis ​​dependentes em X e a variável independente em Y
# Separação é essencial para o treinamento do modelo
x=df.drop(['price'],axis=1)
y=df['price']

In [ ]:
# Dividindo os dados em conjunto de treinamento e conjunto de teste
# Separação feita para o treinamento do modelo e para seus eventuais testes
# X = Df sem a coluna price
# Y = Df com a coluna price

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
# Escalando os valores para converter os valores int para linguagens de máquina
# Normalização dos dados, define que os dados deverão estar em uma escalada de 0 a 1, podendo ser 0.5 ou 0.6.

mmscaler=MinMaxScaler(feature_range=(0,1))
x_train=mmscaler.fit_transform(x_train)
x_test=mmscaler.transform(x_test)
x_train=pd.DataFrame(x_train)
x_test=pd.DataFrame(x_test)

In [ ]:
# Cria um dataframe chamada Results

a={'Model Name':[], 'Mean_Absolute_Error_MAE':[] ,'Adj_R_Square':[] ,'Root_Mean_Squared_Error_RMSE':[] ,'Mean_Absolute_Percentage_Error_MAPE':[] ,'Mean_Squared_Error_MSE':[] ,'Root_Mean_Squared_Log_Error_RMSLE':[] ,'R2_score':[]}
Results=pd.DataFrame(a)
Results.head()

In [ ]:
# Crie objetos de modelos de Regressão/Regressor com hiperparâmetros padrão
# Modelos de regresão preveem valores contínuos, como o preço de passagens aéreas.
# Modelos de classificação preveem como se um cliente vai comprar ou não uma passagem.
modelmlg = LinearRegression()
modeldcr = DecisionTreeRegressor()
modelrfr = RandomForestRegressor()
modelETR = ExtraTreesRegressor()

# Matriz de avaliação para todos os algoritmos
MM = [modelmlg, modeldcr, modelrfr, modelETR]

for models in MM:

    # Ajuste o modelo com dados de trem
    models.fit(x_train, y_train)

    # Preveja o modelo com dados de teste
    y_pred = models.predict(x_test)

    # Imprime o nome do modelo
    print('Nome do Modelo: ', models)

    # Métricas de avaliação para análise de regressão
    print('Erro Absoluto Médio (MAE):', round(metrics.mean_absolute_error(y_test, y_pred),3))
    # Mede a média dos erros absolutos entre os valores reais e os valores previstos pelo modelo.
    # Exemplo: Se o preço real for R$ 300 e a previsão for R$ 320, o erro é R$ 20.
    # Um MAE menor indica previsões mais precisas.

    print('Erro Quadrático Médio (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))
    # Mede a média dos quadrados das diferenças entre os valores reais e previstos. Como eleva ao quadrado os erros, penaliza erros maiores mais severamente.
    # MSE baixo é desejado, mas é mais sensível a grandes discrepâncias.
    # É útil para identificar se há previsões muito erradas.

    print('Raiz do Erro Quadrático Médio (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))
    # É a raiz quadrada do MSE. Retorna o erro médio na mesma unidade da variável prevista (neste caso, o preço das passagens).
    # Mais intuitivo que o MSE, pois mantém a unidade dos dados.
    # Também penaliza erros grandes, mas é fácil de interpretar diretamente.

    print('Coeficiente de Determinação (R²):', round(metrics.r2_score(y_test, y_pred),6))
    # Mede a proporção da variabilidade dos dados explicada pelo modelo.
    # Vai de 0 a 1, onde:
      # 0 significa que o modelo não explica nada.
      # 1 significa que o modelo explica toda a variabilidade dos dados.
    # Um R² mais alto indica um modelo melhor para explicar os dados.
    # No entanto, não considera o número de variáveis no modelo.

    print('Raiz do Erro Quadrático Médio Logarítmico (RMSLE):', round(np.log(np.sqrt(metrics.mean_squared_error(y_test, y_pred))),3))
    # Mede a diferença entre os logaritmos dos valores previstos e reais.
    # É útil para problemas em que os valores têm uma ampla gama de variações e é mais tolerante a erros em valores altos.
    # Um RMSLE menor indica previsões melhores.
    # Penaliza mais erros relativos em valores baixos.

    # Defina a função para calcular o MAPE - Erro Percentual Absoluto Médio
    def MAPE (y_test, y_pred):
        y_test, y_pred = np.array(y_test), np.array(y_pred)
        return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Avaliação do MAPE
    result = MAPE(y_test, y_pred)
    print('Erro Percentual Absoluto Médio (MAPE):', round(result, 2), '%')
    # Mede a porcentagem média de erro em relação aos valores reais.
    # É útil para avaliar o desempenho do modelo em termos percentuais, independentemente das unidades da variável.
    # Um MAPE de 10% significa que, em média, as previsões estão 10% fora dos valores reais.

    # Calcular valores de R quadrado ajustado
    r_squared = round(metrics.r2_score(y_test, y_pred),6)
    adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
    print('R² Ajustado: ', adjusted_r_squared)
    # É uma versão ajustada do R² que penaliza o uso de muitas variáveis no modelo.
    # Útil em datasets com muitas variáveis, pois evita que modelos complexos pareçam melhores do que são.

    print('------------------------------------------------------------------------------------------------------------')

    #-------------------------------------------------------------------------------------------
    new_row = {'Model Name' : models,
               'Mean_Absolute_Error_MAE' : metrics.mean_absolute_error(y_test, y_pred),
               'Adj_R_Square' : adjusted_r_squared,
               'Root_Mean_Squared_Error_RMSE' : np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
               'Mean_Absolute_Percentage_Error_MAPE' : result,
               'Mean_Squared_Error_MSE' : metrics.mean_squared_error(y_test, y_pred),
               'Root_Mean_Squared_Log_Error_RMSLE': np.log(np.sqrt(metrics.mean_squared_error(y_test, y_pred))),
               'R2_score' : metrics.r2_score(y_test, y_pred)}

    Results = pd.concat([Results, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
# Exibir resultados finais
print(Results)

In [ ]:
models=['LinearRegression','DecisionTreeRegressor','RandomForestRegressor','KNeighborsRegressor','ExtraTreesRegressor','GradientBoostingRegressor','XGBRegressor','BaggingRegressor','Ridge Regression','Lasso Regression']
result=pd.DataFrame({'Model_Name':models})
result['Adj_R_Square']=Results['Adj_R_Square']
result['Mean_Absolute_Error_MAE']=Results['Mean_Absolute_Error_MAE']
result['Root_Mean_Squared_Error_RMSE']=Results['Root_Mean_Squared_Error_RMSE']
result['Mean_Absolute_Percentage_Error_MAPE']=Results['Mean_Absolute_Percentage_Error_MAPE']
result['Mean_Squared_Error_MSE']=Results['Mean_Squared_Error_MSE']
result['Root_Mean_Squared_Log_Error_RMSLE']=Results['Root_Mean_Squared_Log_Error_RMSLE']
result['R2_score']=Results['R2_score']
result=result.sort_values(by='Adj_R_Square',ascending=False).reset_index(drop=True)
result

In [ ]:
# Treinando o modelo com
modelETR.fit(x_train, y_train)

# Preveja o modelo com dados de teste
y_pred = modelETR.predict(x_test)

In [ ]:
out=pd.DataFrame({'Price_actual':y_test,'Price_pred':y_pred})
result=df_bk.merge(out,left_index=True,right_index=True)

In [ ]:
result.sample(10)

In [ ]:
plt.figure(figsize=(20,8))
sns.lineplot(data=result,x='days_left',y='Price_actual',color='red')
sns.lineplot(data=result,x='days_left',y='Price_pred',color='blue')
plt.title('Dias restantes para a partida versus preço real do bilhete e preço previsto do bilhete',fontsize=20)
plt.legend(labels=['Price actual','Price predicted'],fontsize=19)
plt.xlabel('Dias restantes para a partida',fontsize=15)
plt.ylabel('Preço real e previsto',fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))  # Define o tamanho do gráfico
sns.regplot(x='Price_actual', y='Price_pred', data=result, color='cyan')  # Cria um gráfico de regressão com os dados reais e previstos
plt.title('Preço Real Vs Preço Previsto', fontsize=20)  # Adiciona um título ao gráfico
plt.xlabel('Preço Real', fontsize=15)  # Define o rótulo do eixo X
plt.ylabel('Preço Previsto', fontsize=15)  # Define o rótulo do eixo Y
plt.show()  # Exibe o gráfico